In [1]:
import pandas as pd
import numpy as np
import torch
import tqdm
import os
import random
from torch.utils.data import Dataset, DataLoader
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

2021-10-24 10:02:04.669615: I tensorflow/stream_executor/platform/default/dso_loader.cc:54] Successfully opened dynamic library libcudart.so.11.0


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="4"
tf_device='/gpu:0'

In [3]:
tokenizer = BertTokenizer.from_pretrained("indolem/indobertweet-base-uncased")
config = BertConfig.from_pretrained("indolem/indobertweet-base-uncased")
config.num_labels = 3
model = BertForSequenceClassification.from_pretrained("indolem/indobertweet-base-uncased", config=config)

Some weights of the model checkpoint at indolem/indobertweet-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/ind

In [4]:
class DocumentSentimentDataset(Dataset):
    # Static constant variable (We need to have this part to comply with IndoNLU standard)
    LABEL2INDEX = {'negative': 0, 'neutral': 1, 'positive': 2} # Label string to index
    INDEX2LABEL = {0:'negative', 1:'neutral', 2: 'positive'} # Index to label string
    NUM_LABELS = 3 # Number of label
   
    def load_dataset(self, path):
        df = pd.read_csv(path, sep='\t', header=None) # Read tsv file with pandas
        df.columns = ['text','sentiment'] # Rename the columns
        df['sentiment'] = df['sentiment'].apply(lambda lab: self.LABEL2INDEX[lab]) # Convert string label into index
        return df
   
    def __init__(self, dataset_path, tokenizer, *args, **kwargs):
        self.data = self.load_dataset(dataset_path) # Load the tsv file

        # Assign the tokenizer for tokenization
        # here we use subword tokenizer from HuggingFace
        self.tokenizer = tokenizer
    
    def __getitem__(self, index):
        data = self.data.loc[index,:] # Taking data from a specific row from Pandas
        text, sentiment = data['text'], data['sentiment'] # Take text and sentiment from the row
        subwords = self.tokenizer.encode(text) # Tokenize the text with tokenizer
        return np.array(subwords), np.array(sentiment), data['text']
   
    def __len__(self):
        return len(self.data)  # Return the length of the dataset

In [5]:
class DocumentSentimentDataLoader(DataLoader):
    def __init__(self, max_seq_len=512, *args, **kwargs):
        super(DocumentSentimentDataLoader, self).__init__(*args, **kwargs)
        self.max_seq_len = max_seq_len # Assign max limit of the sequence length
        self.collate_fn = self._collate_fn # Assign the collate_fn function with our function
       
    def _collate_fn(self, batch):
        batch_size = len(batch) # Take the batch size
        max_seq_len = max(map(lambda x: len(x[0]), batch)) # Find maximum sequence length from the batch 
        max_seq_len = min(self.max_seq_len, max_seq_len) # Compare with our defined limit
       
    # Create buffer for subword, mask, and sentiment labels, initialize all with 0
        subword_batch = np.zeros((batch_size, max_seq_len), dtype=np.int64)
        mask_batch = np.zeros((batch_size, max_seq_len), dtype=np.float32)
        sentiment_batch = np.zeros((batch_size, 1), dtype=np.int64)
       
    # Fill all of the buffer
        seq_list = []
        for i, (subwords, sentiment, raw_seq) in enumerate(batch):
            subwords = subwords[:max_seq_len]
            subword_batch[i,:len(subwords)] = subwords
            mask_batch[i,:len(subwords)] = 1
            sentiment_batch[i,0] = sentiment
            
            seq_list.append(raw_seq)
           
    # Return the subword, mask, and sentiment data
        return subword_batch, mask_batch, sentiment_batch, seq_list

In [6]:
def forward_sequence_classification(model, batch_data, i2w, is_test=False, device = 'cuda', **kwargs):
    # Unpack batch data
    if len(batch_data) == 3:
        (subword_batch, mask_batch, label_batch) = batch_data
        token_type_batch = None
    elif len(batch_data) == 4:
        (subword_batch, mask_batch, token_type_batch, label_batch) = batch_data
    
    # Prepare input & label
    subword_batch = torch.LongTensor(subword_batch)
    mask_batch = torch.FloatTensor(mask_batch)
    token_type_batch = torch.LongTensor(token_type_batch) if token_type_batch is not None else None
    label_batch = torch.LongTensor(label_batch)
            
    if device == "cuda":
        subword_batch = subword_batch.cuda()
        mask_batch = mask_batch.cuda()
        token_type_batch = token_type_batch.cuda() if token_type_batch is not None else None
        label_batch = label_batch.cuda()
    
    # Forward model
    outputs = model(subword_batch, attention_mask=mask_batch, token_type_ids=token_type_batch, labels=label_batch)
    print(outputs)
    loss, logits = outputs[:2]
    
    # generate prediction & label list
    list_hyp = []
    list_label = []
    hyp = torch.topk(logits, 1)[1]
    for j in range(len(hyp)):
        list_hyp.append(i2w[hyp[j].item()])
        list_label.append(i2w[label_batch[j][0].item()])
    return loss, list_hyp, list_label

In [7]:
def document_sentiment_metrics_fn(list_hyp, list_label):
    metrics = {}
    metrics["ACC"] = accuracy_score(list_label, list_hyp)
    metrics["F1"] = f1_score(list_label, list_hyp, average='macro')
    metrics["REC"] = recall_score(list_label, list_hyp, average='macro')
    metrics["PRE"] = precision_score(list_label, list_hyp, average='macro')
    return metrics

In [8]:
import itertools
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
set_seed(241021)

In [9]:
w2i, i2w = DocumentSentimentDataset.LABEL2INDEX, DocumentSentimentDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'negative': 0, 'neutral': 1, 'positive': 2}
{0: 'negative', 1: 'neutral', 2: 'positive'}


In [10]:
text =  'satu satu aku sayang ibu, dua dua juga sayang ayah, tiga-tiga sayang adik kakak, satu dua tiga sayang semuanya'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
print(subwords)
logits = model(subwords)[0]
print(logits)
print(torch.topk(logits, k=1))
label = torch.topk(logits, k=1)[1].squeeze().item()
print(f'Text: {text} | Label : {i2w[label]} ({torch.nn.functional.softmax(logits, dim=-1).squeeze()[label] * 100}%)')

tensor([[   3, 1713, 1713, 2254, 5458, 2477,   16, 1854, 1854, 1614, 5458, 3095,
           16, 2139,   17, 2139, 5458, 4962, 6962,   16, 1713, 1854, 2139, 5458,
         4398,    4]])
tensor([[0.1063, 0.3589, 0.2616]], grad_fn=<AddmmBackward0>)
torch.return_types.topk(
values=tensor([[0.3589]], grad_fn=<TopkBackward0>),
indices=tensor([[1]]))
Text: satu satu aku sayang ibu, dua dua juga sayang ayah, tiga-tiga sayang adik kakak, satu dua tiga sayang semuanya | Label : neutral (37.25643539428711%)


In [11]:
print(f'Text: {text} | Label : {i2w[label]} ({torch.nn.functional.softmax(logits, dim=-1).squeeze()[label] * 100}%)')

Text: satu satu aku sayang ibu, dua dua juga sayang ayah, tiga-tiga sayang adik kakak, satu dua tiga sayang semuanya | Label : neutral (37.25643539428711%)


In [12]:
train = DocumentSentimentDataset('dataset/train_preprocess.tsv', tokenizer)
valid = DocumentSentimentDataset('dataset/valid_preprocess.tsv', tokenizer)
train_loader = DocumentSentimentDataLoader(dataset=train, max_seq_len=100, batch_size=32, num_workers=1, shuffle=True)
valid_loader = DocumentSentimentDataLoader(dataset=valid, max_seq_len=100, batch_size=32, num_workers=1, shuffle=True)

In [13]:
torch.cuda.empty_cache()

In [14]:
def count_param(module, trainable=True):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
count_param(model)

110560515

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-6)
model = model.cuda()

In [16]:
n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)
 
    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm.tqdm(iter(train_loader), leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

(Epoch 1) TRAIN LOSS:1.1632 LR:0.00000300:   4%|█▋                                       | 1/25 [00:00<00:11,  2.16it/s]

SequenceClassifierOutput(loss=tensor(1.1632, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 3.3615e-02,  3.5671e-01, -1.2313e-01],
        [ 1.5387e-01,  4.2158e-01, -1.7582e-01],
        [-6.1776e-02,  2.8236e-01, -2.6932e-01],
        [-1.3965e-01,  2.6802e-01, -9.1601e-03],
        [ 1.3911e-01,  3.6378e-01,  3.1713e-01],
        [ 2.8474e-01,  1.0744e-01,  1.5907e-01],
        [ 4.1207e-01,  2.3071e-02, -3.6158e-02],
        [ 4.2197e-01,  1.6374e-01,  1.5196e-01],
        [ 1.2188e-01,  4.4394e-01, -1.5308e-01],
        [ 8.6493e-02,  3.1165e-01,  3.0392e-01],
        [ 2.1207e-01,  2.7171e-01,  5.5797e-02],
        [-5.1121e-02,  3.3995e-01,  4.7404e-02],
        [ 1.9331e-01,  1.0391e-01,  1.6870e-02],
        [ 1.5999e-01,  3.6368e-01,  2.0139e-01],
        [-3.1938e-04,  3.3215e-01, -3.2678e-02],
        [ 1.1143e-02,  3.8876e-01,  5.1317e-02],
        [ 2.2058e-01,  1.7777e-02,  1.1571e-01],
        [-7.8299e-02,  1.1653e-02,  3.1399e-01],
        [ 7.7959e-02

(Epoch 1) TRAIN LOSS:1.1261 LR:0.00000300:   8%|███▎                                     | 2/25 [00:00<00:07,  3.20it/s]

SequenceClassifierOutput(loss=tensor(1.0889, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 0.3122,  0.3217, -0.1440],
        [ 0.1463, -0.0136,  0.2308],
        [ 0.1727, -0.0009,  0.1943],
        [-0.1019,  0.5760,  0.1275],
        [ 0.3802,  0.5496,  0.2276],
        [ 0.1621,  0.1478,  0.0158],
        [-0.0870,  0.1240, -0.0224],
        [-0.1019,  0.4882,  0.2198],
        [ 0.4551,  0.2383,  0.1481],
        [ 0.3522,  0.2099,  0.2823],
        [ 0.3960,  0.5070,  0.0869],
        [ 0.0337,  0.4887,  0.2073],
        [ 0.1098,  0.2982,  0.0875],
        [ 0.0508,  0.1102,  0.6286],
        [ 0.1664,  0.3772, -0.0606],
        [ 0.1752,  0.2354,  0.1167],
        [ 0.0652,  0.4043,  0.1094],
        [ 0.1208,  0.3034,  0.0591],
        [ 0.1132,  0.4409,  0.1675],
        [-0.0445,  0.1736, -0.1837],
        [ 0.1837, -0.2312, -0.0061],
        [-0.0467,  0.1526, -0.1352],
        [-0.0084,  0.0827,  0.0168],
        [ 0.2302,  0.4138,  0.1310],
        [ 0.19

(Epoch 1) TRAIN LOSS:1.1143 LR:0.00000300:  12%|████▉                                    | 3/25 [00:00<00:05,  3.74it/s]

...

SequenceClassifierOutput(loss=tensor(0.2222, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-0.7593, -1.7014,  2.6840],
        [-1.0556, -1.2331,  2.2801],
        [-0.4463, -1.9238,  2.2743],
        [ 2.3666, -0.4371, -0.9243],
        [ 0.2176,  0.7629, -0.4441],
        [-0.9501, -1.4784,  1.9500],
        [-0.8143, -1.6836,  2.5572],
        [-1.0624, -1.5287,  2.1900],
        [-0.5993, -1.5207,  2.5075],
        [-0.6684, -1.4846,  2.3416],
        [ 0.9272, -1.5966,  0.6404],
        [ 0.8095, -0.1139, -0.3778],
        [ 0.5753,  0.2949, -0.0052],
        [-0.7647, -1.5445,  2.4138],
        [ 1.7884, -0.6643, -0.8930],
        [ 1.2789, -0.5516, -0.2849],
        [-0.7480, -1.9532,  2.4277],
        [-1.0281, -1.9058,  2.5153],
        [ 1.6887, -1.0337, -0.9989],
        [-1.2005, -1.6476,  2.7367],
        [ 2.1977, -0.2597, -1.1488],
        [ 1.8828, -0.6325, -0.7564],
        [-1.0299, -1.6052,  2.3500],
        [-0.1262,  1.5366, -0.5841],
        [-0.64

(Epoch 10) TRAIN LOSS:0.3199 LR:0.00000300:  96%|█████████████████████████████████████▍ | 24/25 [00:06<00:00,  2.68it/s]

SequenceClassifierOutput(loss=tensor(0.2559, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 0.7254,  0.0990, -0.4878],
        [ 0.2523, -1.3308,  1.3309],
        [-1.3066, -1.1074,  2.7711],
        [-1.1408, -1.6297,  2.6447],
        [-0.2022, -0.2565,  0.3066],
        [-0.8928, -1.5844,  2.6296],
        [ 0.0105, -0.1805,  0.4547],
        [-0.6614, -0.7295,  2.0005],
        [ 1.2113,  0.0264, -0.9041],
        [-1.2164, -1.5238,  2.7896],
        [-1.0757, -1.5189,  2.1256],
        [-1.3953, -0.8725,  2.4782],
        [ 1.8007, -0.2554, -0.8710],
        [-0.3457, -0.1102,  0.4699],
        [-1.2518, -1.8067,  2.5920],
        [-1.0845, -1.7266,  2.6442],
        [ 2.2348, -0.5817, -1.1785],
        [ 0.1511, -1.8020,  1.7730],
        [ 1.7021, -0.6425, -0.9898],
        [-0.8346, -1.9479,  2.3100],
        [-0.7446, -1.6225,  2.3538],
        [-0.9127, -1.6759,  2.5642],
        [ 1.0551, -0.6367, -0.0240],
        [-1.1320, -1.4569,  2.7415],
        [-0.56

(Epoch 10) TRAIN LOSS:0.3174 LR:0.00000300: 100%|███████████████████████████████████████| 25/25 [00:06<00:00,  3.71it/s]

(Epoch 10) TRAIN LOSS:0.3174 ACC:0.91 F1:0.87 REC:0.85 PRE:0.92 LR:0.00000300


In [17]:
# Evaluate on validation
model.eval()
torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm.tqdm(iter(valid_loader), leave=True, total=len(valid_loader))
for i, batch_data in enumerate(pbar):
    batch_seq = batch_data[-1]
    loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
    
    # Calculate total loss
    valid_loss = loss.item()
    total_loss = total_loss + valid_loss

    # Calculate evaluation metrics
    list_hyp += batch_hyp
    list_label += batch_label
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)

    pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))
    
metrics = document_sentiment_metrics_fn(list_hyp, list_label)
print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
    total_loss/(i+1), metrics_to_string(metrics)))

VALID LOSS:0.4129 ACC:0.84 F1:0.80 REC:0.77 PRE:0.84:  29%|████████▊                      | 2/7 [00:00<00:00,  6.79it/s]

SequenceClassifierOutput(loss=tensor(0.5209, device='cuda:0'), logits=tensor([[ 1.6022,  0.0610, -0.6325],
        [ 1.5415, -0.7153,  0.0057],
        [ 0.0251, -1.4754,  1.5961],
        [-0.5106, -1.8198,  2.1493],
        [ 0.6256, -2.3428,  1.6935],
        [ 1.9691, -0.7293, -0.5360],
        [ 1.2407, -0.6683,  0.0056],
        [ 1.1910, -0.8565,  0.2248],
        [-0.1905, -0.4875,  1.1694],
        [ 1.5163,  0.2109, -0.4007],
        [ 1.9952, -0.3252, -1.1078],
        [ 0.1421,  0.7415,  0.1316],
        [-0.0185, -0.4474,  0.9149],
        [ 1.7116,  0.0200, -0.8571],
        [ 2.3021, -0.8258, -0.7692],
        [ 0.3830, -0.0900, -0.5695],
        [-1.0319, -1.2908,  2.3917],
        [-0.4671, -1.9519,  2.4526],
        [-0.1232, -0.6399,  1.1979],
        [ 0.0417,  0.3995,  0.2068],
        [ 0.6060,  1.4272, -0.7496],
        [ 1.5699, -1.2414, -0.3771],
        [-0.3993, -1.7767,  2.0441],
        [-0.0807, -0.0514, -0.2329],
        [ 1.2252,  0.2961, -0.3163],
     

VALID LOSS:0.3638 ACC:0.88 F1:0.85 REC:0.83 PRE:0.88:  57%|█████████████████▋             | 4/7 [00:00<00:00,  6.47it/s]

SequenceClassifierOutput(loss=tensor(0.2174, device='cuda:0'), logits=tensor([[ 0.2917,  1.2946, -0.5943],
        [-0.9586, -1.8916,  2.7693],
        [ 2.1209, -0.6995, -0.5268],
        [-0.2695, -0.6276,  1.0945],
        [-1.2645, -1.7874,  2.9435],
        [ 1.8473, -0.5752, -1.0842],
        [-1.4841, -1.4897,  2.8823],
        [-1.3052, -1.7162,  3.1231],
        [ 0.0418, -0.1484,  0.3796],
        [ 1.1245, -0.5135, -0.2867],
        [-1.1683, -1.4840,  2.5991],
        [-1.1683, -1.7737,  2.9686],
        [-0.9982, -1.6459,  2.5148],
        [ 2.4516, -0.9369, -0.7569],
        [-1.3920, -1.5936,  2.8308],
        [-0.7937, -1.6749,  2.3265],
        [-0.9768, -1.8861,  2.7870],
        [-0.9322, -1.7926,  2.6087],
        [-1.3273, -1.7056,  2.7538],
        [ 1.6630, -1.1063, -0.0391],
        [-1.2704, -1.6034,  2.9662],
        [-1.4786, -1.4340,  2.9404],
        [ 0.9151, -0.2246, -0.1893],
        [ 1.9432, -0.8665, -0.5152],
        [ 1.8698, -0.7514, -0.6658],
     

VALID LOSS:0.3218 ACC:0.91 F1:0.89 REC:0.88 PRE:0.90:  86%|██████████████████████████▌    | 6/7 [00:00<00:00,  6.76it/s]

SequenceClassifierOutput(loss=tensor(0.1935, device='cuda:0'), logits=tensor([[ 2.2209, -0.4182, -1.1661],
        [ 2.1915, -0.7597, -1.0946],
        [-1.3349, -1.5440,  2.7933],
        [-0.9303, -1.1808,  2.4547],
        [-1.3404, -1.4546,  2.6862],
        [-0.3688, -1.4697,  2.0464],
        [ 1.7009, -0.5646, -0.5219],
        [-1.1828, -1.4538,  2.5593],
        [-1.0880, -1.7102,  2.6170],
        [ 1.6788, -0.6300, -0.6948],
        [ 2.2376, -0.8655, -0.3439],
        [-1.2149, -1.5052,  2.8163],
        [-0.9936, -2.0224,  2.6162],
        [-0.4291, -0.9042,  1.6399],
        [ 1.1135, -0.2687, -0.3817],
        [ 1.6848, -0.9922, -0.3020],
        [ 1.1803, -0.8877, -0.0915],
        [-1.1092, -1.4820,  2.6541],
        [-0.1325, -1.6265,  2.0049],
        [-0.5748,  1.1407,  0.1592],
        [-0.9847, -1.8617,  2.9025],
        [ 1.7903, -0.9908, -0.4954],
        [-1.0382, -1.8737,  2.6168],
        [ 1.7707, -0.1145, -0.8552],
        [ 1.9438, -0.2474, -1.5069],
     

VALID LOSS:0.3368 ACC:0.91 F1:0.89 REC:0.88 PRE:0.90: 100%|███████████████████████████████| 7/7 [00:01<00:00,  6.29it/s]

SequenceClassifierOutput(loss=tensor(0.4266, device='cuda:0'), logits=tensor([[ 1.4578,  0.5876, -1.0129],
        [ 1.0917, -0.5079, -0.5128],
        [ 0.7812, -1.1402,  0.1224],
        [-1.3073, -1.8274,  3.0044],
        [ 0.7625, -1.3047,  0.8563],
        [ 2.1716, -0.3405, -1.2770],
        [ 0.7731, -0.0829,  0.1464],
        [-1.1277, -1.8785,  2.9902]], device='cuda:0'), hidden_states=None, attentions=None)
(Epoch 10) VALID LOSS:0.3368 ACC:0.91 F1:0.89 REC:0.88 PRE:0.90
